In [1]:
import scipy.io as sio 
import numpy as np
import os
import zipfile
import torch
from torch import nn
from torch.nn import functional as F
from copy import deepcopy,copy
from torch.autograd import Variable
from torch.utils.data import Dataset, DataLoader


In [2]:
data = sio.loadmat('D:/deeplearn/metaDataMine_MLP.mat')
# data = np.array(data['data'])#是L不是1
# data = data.reshape(92,40,1444)
# print(data.shape)
# label = sio.loadmat('D:/deeplearn/label_leibiao_meta.mat')
# label = np.array(label['label'])
# label = np.linspace(0,3679,3680)
# print(label)
e = torch.randperm(460)

train_data = np.array(data['train_data'])#,[40:60]]#[0:20;40:60]
train_label = np.array(data['train_label'])
train_data = train_data
train_label = train_label
print(train_data.shape)

# print(train_label)

train_data = torch.Tensor(train_data)
train_label = torch.LongTensor(train_label)
train_label = train_label.squeeze()
test_data = torch.Tensor(data['test_data'])
test_label = torch.LongTensor(data['test_label'])
test_label = test_label.squeeze()
# c = np.append(w[0:100,:],q[100:1000,:],axis=0)#[c[:,0:250],f[:,0:250]]
# c = np.array(c)
# print(test_data.size)
# print(test_label)
# f = torch.Tensor(f[e])
# c = c.t()
# c = np.array(c)
# d = label['label']
# g = torch.LongTensor(d)
# g = g.squeeze()
# # d = np.array(label['label'])
# d = torch.LongTensor(d[e])
# d = d.squeeze() 
# # datasets = {"train": c, "train_label": d}
# print(d)
# data
# print(np.array(data['E']).shape)
# print(d.shape)

(840, 1444)


In [3]:
class Net(nn.Module):
    def __init__(self, in_dim, out):
        super(Net,self).__init__()
       
        self.fc1 = nn.Linear(in_dim,800)
        self.fc2 = nn.Linear(800,400)
        self.fc3 = nn.Linear(400,200)
#         self.fc3 = nn.Linear(800,200)
#         self.fc2 = nn.Linear(300,100)
        self.fc4 = nn.Linear(200,out)
    def forward(self,x):
        x = F.relu(self.fc1(x))
#         running_mean = nn.Parameter(torch.zeros(800), requires_grad= False)
#         running_var = nn.Parameter(torch.zeros(800), requires_grad= False)
#         x = F.batch_norm(x,running_mean=running_mean,running_var=running_var)
#         x = F.dropout(x,0.3)
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
#         x = F.relu(self.fc3(x))
        x = F.dropout(x,0.3)
#         x = F.batch_norm(x,0.1,0.1)
        x = self.fc4(x)
        return x

In [4]:
in_dim, out, p = 1444,92,0
model = Net(in_dim,out)
cp = []
ap = []
ao = []
co = []
loss = nn.CrossEntropyLoss()
lr, weight_decay,momentum = 0.001,0.0001,0.9
optimizer = torch.optim.Adam(model.parameters(),lr = lr,weight_decay=weight_decay)
# @torchsnooper.snoop()
for t in range(2000): 
    p = 0
    cp = []
    for i in range(56):
        p=p+1
        model.train()
        y_pred= model(train_data[(15*(p-1)):(15*p)].float())
#         print(y_pred.shape)
        lossp = loss(y_pred, train_label[(15*(p-1)):(15*p)].long())
        optimizer.zero_grad()
        lossp.backward()
        optimizer.step()
        _, pred = torch.max(y_pred, 1)#1是每行的最大值
#         print(pred)
#         print(train_label[(40*(p-1)):(40*p)].long())
        num_correct = (pred == train_label[(15*(p-1)):(15*p)].long()).sum()
        accuracy = (pred == train_label[(15*(p-1)):(15*p)].long()).float().mean()
        with torch.no_grad():
            accuracy = np.array(accuracy)
            cp.append(accuracy)
            ao.append(np.array(lossp))
    a = np.array(cp).mean()
    b = np.array(ao).mean()
    
    ap.append(accuracy)
    co.append(b)
#     print(a.shape)
#     if t % 2 == 0:
        
#         model.eval()
#         y_pred = model(test_data)
#         _, pred = torch.max(y_pred, 1)
# #         print("pred:",pred)
# #         print("testlabel",test_label)
# #         print(pred,d)
#         acc = (pred == test_label.long()).float().mean()
#         print("trainAcc:%.6f, %.6f, loss:%.6f}"%(accuracy,acc,lossp))
#         print(accuracy)
sio.savemat('mlpmeta.mat',{'ap':ap,'co':co})
# torch.save(model,'D:/d2l-zh/model.pth') 


In [5]:
# sio.savemat('metaSAE.mat',{'PP':cp})